In [1]:
import numpy as np
import collections as cl
import pandas as pd
filename = "/Users/walfred/Documents/Marklab/figure2/PangenomeAlleles_type.tsv"
HGSVC=["HG00512","HG00513","HG00514","HG00731","HG00732","HG03125","NA12878","NA19238","NA19239","NA24385"]
thefile = "/Users/walfred/Documents/Marklab/allsamples_pops.tsv"

In [2]:
pop_table = pd.read_csv(thefile, sep = '\t', header = None)
pop_table = pop_table.values.tolist()

pop_info = dict()
for row in pop_table:
    pop_info[row[0]] = row[1]
    
pop_info["CHM13"]="EUR"

In [3]:
rows = []
assems_index = cl.defaultdict(int)
allele_index = {}
allassems = set()

lines = {"AFR":[],"NonAFR":[],"All":[]}
with open(filename, mode = 'r') as f:
    onekgheader = f.readline()
    for index, line in enumerate(f):
        line = line.split('\t')
        
        name = line[0]
        ifintron = int(line[5] != 'Exon')
        groupname = name.split('_')[0]
        ifdecoy = int(groupname not in line[2])
        if ifintron or ifdecoy :
            continue
            
        allele_index[line[0]] = index        
        assems = ["_".join(x.split("_")[2:4]) if "_h" in x else "CHM13" if "NC_0609" in x else "HG38" for x in line[-1].split(",")]
   
        AFR_assems = [x for x in assems if x != "HG38" and pop_info[x.split('_')[0]] == 'AFR']
        NoAFR_assems = [x for x in assems if x != "HG38" and pop_info[x.split('_')[0]] != 'AFR']
        assems = list(set([x for x in assems if x != "HG38" ]))
    
        lines['AFR'].append(AFR_assems)
        lines['NonAFR'].append(NoAFR_assems)
        lines['All'].append(assems)
        
        rows.append(assems)
        allassems.update(assems)

allele_counts = [len(x) for x in rows]

In [29]:

lines_counts = dict()
lines_counts['AFR'] = [len(set(x)) for x,y in zip(lines['AFR'],lines['NonAFR']) if len(x) and len(y) == 0]
lines_counts['NonAFR'] = [len(set(y)) for x,y in zip(lines['AFR'],lines['NonAFR']) if len(y) and len(x) == 0]
lines_counts['All'] = [len(set(x)) for x in lines['All']]
lines_counts = {name:[len(set(x)) for x in data if len(x)] for name,data in lines.items()}
numAFR = len([x for x in  lines['AFR'] if len(x)])
numNonAFR = len([x for x in  lines['NonAFR'] if len(x)])
numAll = len([x for x in  lines['All'] if len(x)])


In [30]:
AFR_num = len([x for x in allassems if pop_info[x.split('_')[0]] == 'AFR'])
NonAFR_num = len([x for x in allassems if pop_info[x.split('_')[0]] != 'AFR'])
All_num = len([x for x in allassems])

In [31]:
All_num

231

In [32]:
#!/usr/bin/python

import pandas as pd
import numpy as np
from scipy.special import comb as combination
from scipy.optimize import curve_fit
import math
import collections as cl


#this is the custom instance for larger precision for float number
class precis:
	def __init__(self):
		
		self.num_seg=0
		self.int=0
		self.sign=0
		self.dec=[]
		
	def __str__(self):
		
		compliment=['', '0', '00', '000', '0000', '00000', '000000', '0000000', '00000000', '000000000', '0000000000', '00000000000', '000000000000', '0000000000000', '00000000000000', '000000000000000', '0000000000000000', '00000000000000000']
		
		print_str=''
		for x in self.dec:
			x_str=str(abs(x))
			if len(x_str)<10:
				x_str=compliment[10-len(x_str)]+x_str
			print_str+=x_str
			
		return str(self.int)+'.'+print_str
	
	def __repr__(self):
		return str(self)
	
	def resort(self):
		
		sums=[self.int]
		
		if self.int>0:
			sign=1
		elif self.int<0:
			sign=-1
		else:
			sign=0
			
			
		for x in self.dec:
			
			if x>10000000000:
				
				sums[-1]+=x/10000000000
				x%=10000000000
				
			elif x<-10000000000:
				
				sums[-1]-=x/-10000000000
				x%=-10000000000
				
			if x>0:
				current_sign=1
			elif x<0:
				current_sign=-1
			else:
				current_sign=0
				
			if current_sign*sign<0:
				
				sums[-1]+=-sign
				x=sign*10000000000+x
				
			if not sign:
				sign=current_sign
				
			sums.append(x)
			
			
		self.int=sums[0]
		
		self.dec=sums[1:]
		
		self.sign=sign
		
		
		
class precis_cal(precis):
	
	def __init__(self, x, num_seg=3):
		
		
		self.int=int(x)
		
		if x<0:
			self.sign=-1
		else:
			self.sign=1
			
		dec=x-self.int
		
		self.num_seg=num_seg
		
		thefactor=10**10
		
		allsegs=[]
		
		remainder=dec
		for i in range(num_seg):
			
			remainder*=thefactor
			quotient=int(remainder)
			remainder-=quotient
			allsegs.append(quotient)
			
			
		self.dec=allsegs
		
		
		
		
	def __add__(self,another):
		
		thesum=precis_cal(0,0)
		
		if isinstance(another, precis_cal):
			
			
			if self.num_seg>=another.num_seg:
				addpairs=list(map(sum,list(zip(self.dec,another.dec))+[(x,0) for x in self.dec[:(self.num_seg-another.num_seg)]]))
			else:
				addpairs=list(map(sum,list(zip(self.dec,another.dec))+[(0,x) for x in another.dec[:(another.num_seg-self.num_seg)]]))		
				
			thesum.num_seg=len(addpairs)
			
			thesum.int=another.int+ self.int
			
			thesum.dec=addpairs
			
			thesum.sign=self.sign
			
			thesum.resort()
			
			
			
		elif type(another)==type(0):
			
			
			thesum.int+=another
			
			thesum.num_seg=self.num_seg
			
			thesum.dec=self.dec
			
			thesum.sign=self.sign
			
		else:
			
			thesum=self+precis_cal(another)
			
			
		return thesum
	
	def __radd__(self, another):
		
		if isinstance(another, precis_cal):
			
			return self+another
		
	def __mul__(self, factor):
		
		if isinstance(factor, int):
			
			
			new=precis_cal(0,0)
			new.dec=[x*factor for x in self.dec]
			new.int=self.int*factor
			new.num_seg=self.num_seg
			new.resort()
			
			
			
		return new
	
	
	
	def __neg__(self):
		
		new=precis_cal(0,0)
		new.num_seg=self.num_seg
		new.int=-self.int
		new.sign=-self.sign
		new.dec=[-x for x in self.dec]
		
		return new
	
	def __sub__(self,another):
		
		return self+ -another
	
	
	def __pow__(self, base):
		
		if not len(self.dec):
			return base**self.int
		
		start=base**(float(self.dec[0])/10**10+self.int)
		
		
		for i,x in enumerate(self.dec[1:]):
			
			start*=base**(float(x)/10**(10*(i+2)))
			
		return start
	
	
def combination_log(total,obs):
	
	def sum(sequence):
		
		if not len(sequence):
			return 0
		
		start=sequence[0]
		for value in sequence[1:]:
			start = start + value
		return start
	obs = min(obs, total - obs)
	result=sum([np.log2(np.longdouble(x)) for x in range(total-obs+1, total+1)]) - sum([np.log2(np.longdouble(x)) for x in range(1,obs+1)])
	
	return result

#the instance used for downsampling 

class NUI_projection:
	
	#initiate current sample size, and target down sample sizes
	def __init__(self,current_size):

		self.current_size=current_size
		
	
	#used to find observed numbers of SVs respect to sample count for the SV. 
	def count(self,sample_counts):
		
		counts_data={x:0 for x in range(self.current_size + 1)}
		
		for x in sample_counts:
			counts_data[x]+=1
		
		#count data is storaged here, which is series V at current sample_size, the observed SV numbers at certain count.
		self.obs={count:obs for count, obs in counts_data.items() if count > 0}
		self.current_size = len(self.obs) 
		return self
		
	
	#used to down sampling based on sample count of SVs 
	def downsample(self,outfile=''):
		
		#This is series of A as described in reasoning, A(sample_size,count)=V(sample_size,count)/combination(sample_size,count)
		#here we use log value to make sure it will not break the maximum integet size, to enhance the accuracy (python2 can only extend to 17 digits, but we are not sure if this is enough for log values), we used my own components to extend precisions.
		self.last_Aseries={k:pow(2, -combination_log(self.current_size,k)+np.log2(np.longdouble(obs))) for k,obs in self.obs.items() if k > 0}
		
		df_proj=[[self.obs[count] for count in sorted(self.obs.keys())]]
		
		for down_num in range(1, self.current_size ):
			
			sample_size=self.current_size-down_num
			
			#series A follows binomial rule as described in reasoning. A(sample_size,count)= A(sample_size+1,count)+A(sample_size+1,count+1),
			#we use last A(sample_size+1) series to find current A(sample_size) series
			self.current_Aseries={count: self.last_Aseries[count] +  self.last_Aseries[count+1]  for count in range(1,sample_size+1) if count in self.last_Aseries and count + 1 in self.last_Aseries}
			self.last_Aseries=self.current_Aseries
			
			#find series V at down sample size based on series A
			row=[combination(sample_size,count)*Aseries for count,Aseries in self.current_Aseries.items()]
			df_proj.append(row)
			
		
		#make it to be a dataframe
		self.downsample= pd.DataFrame.from_records(df_proj).T
		
		self.downsample.columns= list(self.obs.keys())[::-1]
		self.downsample.index=list(self.obs.keys())
		
		#save to file
		if len(outfile):
			
			self.downsample.to_csv(outfile,mode='w',sep=',',index=False)
			
		
		#sum up to find all expect NUIs at each sample size 
		self.SVcounts=list(map(sum,df_proj))[::-1]
	
		
		return self
		
	
	def upsample(self,targetsize,tail_size=10,inits=[1000.0,10.00,0.001,0.1]):
		
		#Since alpha>>error>beta, we predict alpha and error together
		def incre_vs_samplesize(x, alpha, beta,celta,gamma):
			
			return  (1-abs(celta)*(x-1))*(abs(alpha)/x)+(1-abs(gamma)*(x-2))*abs(beta)
		
		#+abs(beta)*(x-1)*(1-(x-1)*(abs(gamma)))
		#find increment
		self.SVincre=[self.SVcounts[0]]+[x-y for x,y in zip(self.SVcounts[1:], self.SVcounts)]
		
		
		
		obs_sample_sizes=list(range(self.current_size+1))		
		
		betas=[]
		
		#load x and y values
		xdata=obs_sample_sizes[-tail_size:]
		ydata=self.SVincre[-tail_size:]
		
		#fitting to find parameters
		
		para_fits = curve_fit(incre_vs_samplesize, xdata=xdata, ydata=ydata, p0=inits,method='lm',maxfev=100000)[0]
		
		self.para_fits=para_fits
		
		print ('fitted model:(%f/x)*(1-%f)^(x-1)+%f*(1-%f)^(x-1)'%(abs(para_fits[0]),abs(para_fits[2]), abs(para_fits[1]),abs(para_fits[3])))

		#redefine model 
		def incre_vs_samplesize(x, alpha=abs(para_fits[0]), beta=abs(para_fits[1]), celta=abs(para_fits[2]), gamma=abs(para_fits[3])):
			
			return (1-abs(celta)*(x-1))*(abs(alpha)/x)+(1-abs(gamma)*(x-1))*abs(beta)
		
		
		#projection based on model
		incres_proj=list(map(incre_vs_samplesize,list(range(self.current_size+1,targetsize+1))))
		
		allincres=self.SVincre+incres_proj
		
		self.allSV_proj={}
		
		start=0
		for i,incre in enumerate(allincres):
			
			start+=incre
			self.allSV_proj[1+i]=start
		
		
		return self

#downsample the the datasets
def downsample(allcounts, current_size,outfile=''):
	
	projection=NUI_projection(current_size).count(allcounts).downsample(outfile).downsample
	
	return projection

#upsample project the the datasets
def projection(allcounts, current_size,project_size):
	
	projection=NUI_projection(current_size).count(allcounts).downsample().upsample(project_size).allSV_proj
	
	return projection


In [33]:
print(lines_counts['All'][0])

4


In [35]:
#SVcounts = downsample(lines_counts['AFR'], AFR_num,outfile='/Users/walfred/Documents/Marklab/figure2/AFR_proj.csv')
#SVcounts = downsample(lines_counts['NonAFR'], NonAFR_num,outfile='/Users/walfred/Documents/Marklab/figure2/NonAFR_proj.csv')
SVcounts = downsample(lines_counts['All'], All_num,outfile='/Users/walfred/Documents/Marklab/figure2/All_proj.csv')


In [63]:
proj = projection(allele_counts, 231,701)
[proj[i+1]-proj[i] for i in range(1,701)]

231
{1: 93.290043290043286106, 2: 0.281385281385281405, 3: 0.004377168303502080096, 4: 5.6443644624272792433e-05, 5: 9.0785605905823613787e-07, 6: 1.7666978880493966314e-08, 7: 4.018411347502963207e-10, 8: 1.088305686414548083e-11, 9: 3.332767270153126706e-13, 10: 1.23145136439402844296e-14, 11: 5.168021797738561973e-16, 12: 2.322280374753317522e-17, 13: 1.3011145179684187094e-18, 14: 6.621163416255508926e-20, 15: 4.63533105288423231e-21, 16: 2.8540944793433756e-22, 17: 2.0212041924232039596e-23, 18: 1.6784671279548058103e-24, 19: 1.2498287838475425607e-25, 20: 1.1608973764699254149e-26, 21: 1.1674622868314584424e-27, 22: 9.955104771431060769e-29, 23: 1.1024936415053371969e-29, 24: 1.2400043747578827117e-30, 25: 1.3764255992966253191e-31, 26: 1.9696832895718857491e-32, 27: 2.070539667677511046e-33, 28: 3.0520200182749236254e-34, 29: 4.3126369823450305473e-35, 30: 5.7949153228115562848e-36, 31: 8.0328332530846293654e-37, 32: 1.3373581848378723556e-37, 33: 2.1025229180150193656e-38, 34: 

fitted model:(23195.508337/x)*(1-0.000000)^(x-1)+1.771673*(1-0.024122)^(x-1)


[3482.055448898926485,
 2648.3461544847147255,
 2188.5707939005386615,
 1889.4578096339306468,
 1676.8348367535347272,
 1516.7418993263250364,
 1391.1318256311001029,
 1289.4513519825973145,
 1205.0947656695505223,
 1133.7078541812569359,
 1072.3003727924827153,
 1018.7488564564220681,
 971.5026303191982926,
 929.40210399967199173,
 891.5622243851998032,
 857.2953261185113245,
 826.0586537550211297,
 797.41780907962903413,
 771.02075335347176477,
 746.57897011309724,
 723.85358735608389225,
 702.6449988384660017,
 682.784995777015542,
 664.13072712839564105,
 646.56001035465681426,
 629.9676523180871719,
 614.2625346096610208,
 599.3652836636881531,
 585.2063927398770389,
 571.7246963409390901,
 558.86612191870671396,
 546.5826615294149491,
 534.83151929274606573,
 523.57440038006438954,
 512.7769147096043518,
 502.4080742012245615,
 492.4398667998490815,
 482.8468938468079763,
 473.60606000430937002,
 464.69630699859030898,
 456.0983840743402986,
 447.7946493459076045,
 439.7688972641